## Linear Optimization (pyomo vs. linopy)


### Exercise:


A post office requires a different number of full time employees on different days of the week: Each worker must work 5 consecutive days and then have 2 days off. Minimize the number of workers to be hired.

Sets:
$$D = \text{set of week days}$$
$$W = \text{set of consecutive working days}$$
Variables:
$$x \in \mathbb{Z}^+$$
Objective:
minimize the total number of employees
$$min_x\text{ z} = \sum_{d \in D} x_d$$
subject to:
$$\sum_{w \in W} x_{(d+w)} \mod 7 \geq \epsilon _d \quad \forall d \in D$$
$$\epsilon_d:\text{number of full time employees per day}$$  
with "number of full time employees per day":
<table style="margin-left: auto;margin-right: auto;">
  <tr>
    <td>Monday</td>
    <td>Tuesday</td>
    <td>Wednesday</td>
    <td>Thursday</td>
    <td>Friday</td>
    <td>Saturday</td>
    <td>Sunday</td>
  </tr>
  <tr>
    <td>17</td>
    <td>13</td>
    <td>15</td>
    <td>19</td>
    <td>14</td>
    <td>16</td>
    <td>11</td>
  </tr>
</table>


## Pyomo Solution

In [ ]:
import pyomo.environ as pyo
model = pyo.ConcreteModel()

model.d = pyo.Set(initialize = range(7), doc='days of the week, starting at monday')
model.wd = pyo.Set(initialize = range(5), doc='consecutive working days')

model.x = pyo.Var(model.d, within=pyo.NonNegativeIntegers, doc='number of starting workers per day' )
model.epsilon = pyo.Param(
    model.d,
    initialize={day: employees for day, employees in zip(model.d, [17,13,15,19,14,16,11])},
    doc='number of full time employees per day'
)

model.total_number_employees = pyo.Objective(expr = sum(model.x[d] for d in model.d), sense=pyo.minimize)

def rule1(model, d):
    return sum(model.x[(d+ working_day)%7]  for working_day in model.wd)  >= model.epsilon[d]

model.employees_day_constraint = pyo.Constraint(model.d, rule=rule1, doc="full time employees per day requirement")

optimizer = pyo.SolverFactory('appsi_highs')
result = optimizer.solve(model)

result.write()
print("\n# " + "="*50)
print(f"# Number of overall employees: {model.total_number_employees()}")
print("# " + "="*50 +"\n")
model.pprint()


#### How to find the variables and objective of a model, if you don't know the name:

In [ ]:

for model_var in model.component_objects(pyo.Var, active=True):
    print(model_var.doc)
    varobject = getattr(model, str(model_var))
    for index in varobject:
        print(f"{model_var}[{index}]: {pyo.value(varobject[index])}")
for o in  model.component_objects(pyo.Objective, active=True):
    obj = getattr(model, str(o))
    print(f"{obj.name}: {pyo.value(obj)}")


# Linopy Solution

In [ ]:
import linopy
import xarray as xr
import pandas as pd


In [ ]:
model = linopy.Model()

## Sets
# the days of the week, starting at monday
d = pd.RangeIndex(7) #, name='days')
# consecutive working days
wd = pd.RangeIndex(5)

## Variables
#   x(d)    number of starting workers per day
#   z       total number of employees to be hired
x = model.add_variables(lower=0, coords=[d],name='x', integer=True)


In [ ]:

## Parameters
# number of full time employees per day
epsilon = xr.DataArray([17, 13, 15, 19, 14, 16, 11], coords=[d], name="epsilon")

In [ ]:
## Constraints

def con_lhs(model, d):
    return sum(x[(d+ working_day)%7] for working_day in wd)

con = model.linexpr(con_lhs, [d]) >= epsilon


In [ ]:
model.add_constraints(con, name='full time employee requirement')

In [ ]:
model.constraints

In [ ]:
print(linopy.available_solvers)

In [ ]:
objective = x.sum()
model.add_objective(objective, sense="min")

model.solve()
